#
## Import From CSV Example

In [1]:
# ipywidgets is required to run Plotly in Jupyter Notebook.
# Uncomment and run the following line to install it if required.

#!pip install ipywidgets

In [2]:
import ta

import pandas as pd


%matplotlib inline

In [3]:
def load_csv(filename):
    df = pd.read_csv('data/' + filename, skiprows=1)
    df.drop(columns=['symbol', 'volume_btc'], inplace=True)

    # Fix timestamp form "2019-10-17 09-AM" to "2019-10-17 09-00-00 AM"
    df['date'] = df['date'].str[:14] + '00-00 ' + df['date'].str[-2:]

    # Convert the date column type from string to datetime for proper sorting.
    df['date'] = pd.to_datetime(df['date'])

    # Make sure historical prices are sorted chronologically, oldest first.
    df.sort_values(by='date', ascending=True, inplace=True)

    df.reset_index(drop=True, inplace=True)

    # Format timestamps as you want them to appear on the chart buy/sell marks.
    df['date'] = df['date'].dt.strftime('%Y-%m-%d %I:%M %p')

    return df

In [4]:
df = load_csv('Coinbase_BTCUSD_1h.csv')
df.head()

,date,open,high,low,close,volume
0,2017-07-01 11:00 AM,2505.56,2513.38,2495.12,2509.17,287000.32
1,2017-07-01 12:00 PM,2509.17,2512.87,2484.99,2488.43,393142.50
2,2017-07-01 01:00 PM,2488.43,2488.43,2454.40,2454.43,693254.01
3,2017-07-01 02:00 PM,2454.43,2473.93,2450.83,2459.35,712864.80
4,2017-07-01 03:00 PM,2459.35,2475.00,2450.00,2467.83,682105.41


## Data Preparation
### Create the dataset features,
Now we import all the indicator from TA lib, anyway you can skip this


In [5]:
dataset = ta.add_all_ta_features(df, 'open', 'high', 'low', 'close', 'volume', fillna=True)
dataset.head(3)

c:\users\kit\pycharmprojects\tensortrade\venv\lib\site-packages\ta\trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
c:\users\kit\pycharmprojects\tensortrade\venv\lib\site-packages\ta\trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


,date,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
0,2017-07-01 11:00 AM,2505.56,2513.38,2495.12,2509.17,287000.32,154659.537174,287000.32,0.538883,0.000000e+00,...,-23.055860,0.0,2509.170000,0.0,0.000000,0.000000,0.000000,-65.047642,0.000000,0.000000
1,2017-07-01 12:00 PM,2509.17,2512.87,2484.99,2488.43,393142.50,-141466.449196,-106142.18,-0.207995,-8.153775e+06,...,-87.883057,0.0,2500.178580,0.0,2.871568,0.574314,2.297254,-0.826568,-0.830003,-0.826568
2,2017-07-01 01:00 PM,2488.43,2488.43,2454.40,2454.43,693254.01,-833498.148276,-799396.19,-0.606888,-1.035618e+07,...,-99.949135,0.0,2480.168753,0.0,12.006246,2.860700,9.145546,-1.366323,-1.375743,-2.181598


### Create Chart Price History Data
Note: It is recommended to create the chart data *after* creating and cleaning the dataset to ensure one-to-one mapping between the historical prices data and the dataset.

In [6]:
price_history = dataset[['date', 'open', 'high', 'low', 'close', 'volume']]  # chart data
price_history.head(3)

dataset.drop(columns=['date', 'open', 'high', 'low', 'close', 'volume'], inplace=True)
dataset.head()

,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,...,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
0,1.546595e+05,287000.32,0.538883,0.000000e+00,50.000000,0.000000,0.000000,-187039.681889,1000.000000,2505.890000,...,-23.055860,0.0,2509.170000,0.0,0.000000,0.000000,0.000000,-65.047642,0.000000,0.000000
1,-1.414664e+05,-106142.18,-0.207995,-8.153775e+06,0.000000,-37727.185435,-37727.185435,-189936.530400,1000.000000,2499.843813,...,-87.883057,0.0,2500.178580,0.0,2.871568,0.574314,2.297254,-0.826568,-0.830003,-0.826568
2,-8.334981e+05,-799396.19,-0.606888,-1.035618e+07,0.000000,-135063.834683,-86395.510059,-12721.682056,1000.000000,2482.635850,...,-99.949135,0.0,2480.168753,0.0,12.006246,2.860700,9.145546,-1.366323,-1.375743,-2.181598
3,-1.020509e+06,-86531.39,-0.489157,-8.375687e+06,0.000000,-29277.431008,-67356.150375,-8043.126273,1000.000000,2475.369419,...,-86.378897,0.0,2471.140583,0.0,18.106817,5.909923,12.196893,0.200454,0.200253,-1.985517
4,-7.296597e+05,595574.02,-0.263570,-6.352838e+06,27.438846,439.814720,-50407.159102,3780.909265,1003.448066,2472.636246,...,-71.868097,0.0,2469.726121,0.0,21.283294,8.984598,12.298696,0.344807,0.344213,-1.647557
